## import包

In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

/home/caihengxing/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


## 读数据

In [2]:
os.listdir('./')

['category_tree.csv',
 'events.csv',
 'item_properties_part1.csv',
 'item_properties_part2.csv',
 'zip',
 'Retailrocket_data_process.ipynb',
 '.ipynb_checkpoints']

In [3]:
category_tree = pd.read_csv('./category_tree.csv')
events = pd.read_csv('./events.csv')
item_properties_part1 = pd.read_csv('./item_properties_part1.csv')
item_properties_part2 = pd.read_csv('./item_properties_part2.csv')

In [4]:
category_tree.head(2)

,categoryid,parentid
0,1016,213.0
1,809,169.0


In [5]:
events.head(2)

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN


In [6]:
item_properties_part1.head(2)

,timestamp,itemid,property,value
0,1435460400000,460429,categoryid,1338
1,1441508400000,206783,888,1116713 960601 n277.200


In [7]:
item_properties_part2.head(2)

,timestamp,itemid,property,value
0,1433041200000,183478,561,769062
1,1439694000000,132256,976,n26.400 1135780


## 交互表

In [8]:
events.head()

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN
3,1433221955914,483717,view,253185,NaN
4,1433221337106,951259,view,367447,NaN


In [9]:
events['event'].unique()

array(['view', 'addtocart', 'transaction'], dtype=object)

In [11]:
import datetime
events['time'] = events['timestamp'].apply(
                lambda ts: datetime.datetime.utcfromtimestamp(ts/1000).strftime('%Y-%m-%d %H:%M:%S'))

In [13]:
events.drop('timestamp', axis=1, inplace=True)

In [14]:
events.head()

,visitorid,event,itemid,transactionid,time
0,257597,view,355908,NaN,2015-06-02 05:02:12
1,992329,view,248676,NaN,2015-06-02 05:50:14
2,111016,view,318965,NaN,2015-06-02 05:13:19
3,483717,view,253185,NaN,2015-06-02 05:12:35
4,951259,view,367447,NaN,2015-06-02 05:02:17


In [18]:
events['time'] = pd.to_datetime(events['time'])

In [19]:
events['time'].min(), events['time'].max()

(Timestamp('2015-05-03 03:00:04'), Timestamp('2015-09-18 02:59:47'))

最后七天作为测试集

In [20]:
import datetime
split_time = datetime.datetime.strptime('2015-09-12 00:00:00', '%Y-%m-%d %H:%M:%S')

train = events.loc[events['time'] < split_time]
test = events.loc[events['time'] >= split_time]

In [21]:
train['time'].min(), train['time'].max()

(Timestamp('2015-05-03 03:00:04'), Timestamp('2015-09-11 23:59:51'))

In [22]:
test['time'].min(), test['time'].max()

(Timestamp('2015-09-12 00:00:02'), Timestamp('2015-09-18 02:59:47'))

In [23]:
train.head()

,visitorid,event,itemid,transactionid,time
0,257597,view,355908,NaN,2015-06-02 05:02:12
1,992329,view,248676,NaN,2015-06-02 05:50:14
2,111016,view,318965,NaN,2015-06-02 05:13:19
3,483717,view,253185,NaN,2015-06-02 05:12:35
4,951259,view,367447,NaN,2015-06-02 05:02:17


In [24]:
test.head()

,visitorid,event,itemid,transactionid,time
1351100,404970,view,70148,NaN,2015-09-12 00:16:09
1351102,221450,view,209849,NaN,2015-09-12 00:06:43
1351103,1090257,view,367975,NaN,2015-09-12 00:26:57
1351105,181952,view,343961,NaN,2015-09-12 00:20:45
1351112,71466,view,197550,NaN,2015-09-12 00:14:51


In [25]:
test['event'].unique()

array(['view', 'addtocart', 'transaction'], dtype=object)

计算评价指标的时候, test只用transaction的数据

In [27]:
test = test.loc[test['event'] == 'transaction']

## item表

In [28]:
category_tree['categoryid'] = category_tree['categoryid'].astype(str)

In [29]:
item_properties_part1 = pd.merge(item_properties_part1, category_tree, 
                                 left_on='property', right_on='categoryid', how = 'left')

item_properties_part2 = pd.merge(item_properties_part2, category_tree, 
                                 left_on='property', right_on='categoryid', how = 'left')

In [30]:
item_properties_part1.head()

,timestamp,itemid,property,value,categoryid,parentid
0,1435460400000,460429,categoryid,1338,NaN,NaN
1,1441508400000,206783,888,1116713 960601 n277.200,888,866.0
2,1439089200000,395014,400,n552.000 639502 n720.000 424566,400,110.0
3,1431226800000,59481,790,n15360.000,790,1492.0
4,1431831600000,156781,917,828513,917,1374.0


In [32]:
item_properties_part = item_properties_part1.copy()
item_properties_part = item_properties_part.append(item_properties_part2)

## 存档

In [31]:
output_path = './Retailrocket_AutoX/'
os.makedirs(output_path, exist_ok = True)

In [33]:
train.to_csv(output_path + 'inter_df.csv', index = False)
test.to_csv(output_path + 'test.csv', index = False)

item_properties_part.to_csv(output_path + 'item_df.csv', index = False)